In [1]:
import pandas as pd
import numpy as np
from glob import glob
import folium
from datetime import datetime
from geopy.geocoders import Nominatim

In [2]:
def get_lat_lon(location_name):
    geolocator = Nominatim(user_agent="location_finder")
    location = geolocator.geocode(location_name)
    if location:
        latitude, longitude = location.latitude, location.longitude
        return latitude, longitude
    else:
        return None
     
def generate_map(coordinates, location_name):
    if coordinates:
        lat = coordinates[0]
        long = coordinates[1]
        map = folium.Map(location=[lat, long], zoom_start=150)

        # create an iframe pop-up for the marker
        popup_html = f"<b>Date:</b> 10th October, 2023<br/>"
        popup_html += f"<b>Location:</b> Department of Maths <br/>"
        popup_html += f"<b>Time:</b> {datetime.now()}<br/>"
        popup_html += '<b><a href="{}" target="_blank">Event Page</a></b>'.format(
            "Testing Map"
        )
        popup_iframe = folium.IFrame(width=200, height=110, html=popup_html)

        folium.Marker(
            location=[lat, long],
            popup=folium.Popup(popup_iframe),
            icon=folium.Icon(color="red"),
        ).add_to(map)
        points = [(24.94411, 67.08085), (24.9443, 67.0793), (24.94351, 67.08298)]
        folium.PolyLine(points, color="red", weight=5, opacity=0.85).add_to(map)

        map.save("map.html")
    else:
        print(
            f"Location not found for {location_name}. Please check the spelling or provide more details."
        )

In [3]:
files = glob('./csv_data/*.csv')

In [4]:
files

['./csv_data\\20240125-110332 - Right portion from parking.csv',
 './csv_data\\20240125-111600 - From gate no 2.csv',
 './csv_data\\20240125-112847 - Parking to main library.csv',
 './csv_data\\20240125-114435 - Parking to canteen.csv',
 './csv_data\\20240125-115555 - Main gate to main building.csv']

In [5]:
final = pd.DataFrame()

for file in files:
    file_data = pd.read_csv(files[0])
    final = pd.concat([final,file_data])

In [6]:
# final.to_excel('final.xlsx',index=False)

In [7]:
data = final.copy(deep=True)

In [8]:
data = data.drop_duplicates()

In [9]:
data

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,W,1/25/2024 6:03,24.850678,67.004602,2.1,0.9,NaN,0.155,53.0,16,37,NaN,Parking
1,W,1/25/2024 6:04,24.850440,67.005167,2.2,4.2,NaN,0.000,NaN,20,36,NaN,Talpur house
2,W,1/25/2024 6:06,24.850392,67.004258,2.8,12.3,NaN,0.000,NaN,19,37,NaN,Main library
3,W,1/25/2024 6:09,24.850920,67.003913,2.8,8.1,NaN,1.481,137.0,15,39,NaN,English department
4,W,1/25/2024 6:10,24.850933,67.004303,1.9,0.6,NaN,0.615,96.0,21,39,NaN,Indoor sports hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,T,1/25/2024 6:11,24.850717,67.004200,2.8,-3.5,NaN,0.502,258.0,16,39,NaN,NaN
489,T,1/25/2024 6:11,24.850683,67.004173,2.6,-3.8,NaN,0.507,153.0,16,39,NaN,NaN
490,T,1/25/2024 6:11,24.850650,67.004172,2.4,-4.1,NaN,0.505,167.0,16,39,NaN,NaN
491,T,1/25/2024 6:11,24.850640,67.004175,2.5,-3.9,NaN,0.523,167.0,16,39,NaN,NaN


# Removing Extra Columns

In [10]:
data = data[['type','latitude','longitude','desc']]

In [11]:
data

,type,latitude,longitude,desc
0,W,24.850678,67.004602,Parking
1,W,24.850440,67.005167,Talpur house
2,W,24.850392,67.004258,Main library
3,W,24.850920,67.003913,English department
4,W,24.850933,67.004303,Indoor sports hall
...,...,...,...,...
488,T,24.850717,67.004200,NaN
489,T,24.850683,67.004173,NaN
490,T,24.850650,67.004172,NaN
491,T,24.850640,67.004175,NaN


# Separating Look Up Table

In [12]:
reference = data[data['type'] =='W']

In [13]:
reference

,type,latitude,longitude,desc
0,W,24.850678,67.004602,Parking
1,W,24.850440,67.005167,Talpur house
2,W,24.850392,67.004258,Main library
3,W,24.850920,67.003913,English department
4,W,24.850933,67.004303,Indoor sports hall


# Separating Tracking

In [14]:
tracking = data[data['type'] =='T']

# Marking Land Marks

In [15]:
for lat,long,label in zip(reference.latitude,reference.longitude,reference.desc):
    condition = (tracking['latitude']==lat)&(tracking['longitude']==long)
    print("Marking",label)
    tracking.loc[condition, 'desc'] = label

Marking Parking
Marking Talpur house
Marking Main library
Marking English department
Marking Indoor sports hall


In [16]:
# tracking.to_excel('check_tracking.xlsx',index=False)

In [17]:
def fetch_tracking_records(df, start_location, end_location):
    start_location_df = df[df['desc'] == start_location]
    end_location_df = df[df['desc'] == end_location]
    if start_location_df.empty or end_location_df.empty:
        print(f"One or both of the specified locations not found.")
        return pd.DataFrame()
    start_index = start_location_df.index[0]
    end_index = end_location_df.index[0]
    tracking_records = df.loc[start_index:end_index, ['latitude', 'longitude']]
    return tracking_records

In [18]:
# tracking_memory = {}

# all_locations = list(reference['desc'])
# for fromm in all_locations:
#     temp_locations = all_locations.copy()
#     temp_locations.remove(fromm)
#     for tto in temp_locations:
#         key = f'{fromm}-{tto}'
#         reverse_key = f'{tto}-{fromm}'
#         value = fetch_tracking_records(tracking,fromm,tto)
#         if not value.empty:
#             tracking_memory[key] = value
#             tracking_memory[reverse_key] = value
#         else:
#             pass

tracking_memory = {}

all_locations = list(reference['desc'])
for i, fromm in enumerate(all_locations):
    for j, tto in enumerate(all_locations):
        if i != j:  # Ensure different locations
            key = f'{fromm}-{tto}'
            reverse_key = f'{tto}-{fromm}'
            value = fetch_tracking_records(tracking, fromm, tto)
            if not value.empty:
                tracking_memory[key] = value
                tracking_memory[reverse_key] = value


In [19]:
tracking_memory['Parking-Indoor sports hall']

,latitude,longitude
8,24.850678,67.004602
9,24.850680,67.004602
10,24.850685,67.004605
11,24.850700,67.004612
12,24.850717,67.004622
...,...,...
420,24.850935,67.004270
421,24.850935,67.004282
422,24.850937,67.004293
423,24.850938,67.004300


In [20]:
lat,long = get_lat_lon('Sindh Madressa-tul-Islam University')
smiu_map = folium.Map(location=[lat, long], zoom_start=150)

In [24]:
smiu_map

In [25]:
# locations = folium.map.FeatureGroup()

# for lat, lng in zip(reference.latitude, reference.longitude):
#     locations.add_child(
#         folium.CircleMarker(
#             [lat, lng],
#             radius=15,
#             color='yellow',
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6
#         )
#     )

for lat, lng, label in zip(reference.latitude, reference.longitude, reference.desc):
    folium.Marker(
        [lat, lng],
        popup=label,
    ).add_to(smiu_map)

smiu_map

In [26]:
tracking_memory.keys(
    
)

dict_keys(['Parking-Talpur house', 'Talpur house-Parking', 'Parking-Main library', 'Main library-Parking', 'Parking-English department', 'English department-Parking', 'Parking-Indoor sports hall', 'Indoor sports hall-Parking', 'Talpur house-Main library', 'Main library-Talpur house', 'Talpur house-English department', 'English department-Talpur house', 'Talpur house-Indoor sports hall', 'Indoor sports hall-Talpur house', 'Main library-English department', 'English department-Main library', 'Main library-Indoor sports hall', 'Indoor sports hall-Main library', 'English department-Indoor sports hall', 'Indoor sports hall-English department'])

In [41]:
track = tracking_memory['Indoor sports hall-Parking']

In [42]:
points = [(x, y) for x, y in zip(track.latitude, track.longitude)]
folium.PolyLine(points, color="pink", weight=5, opacity=0.85).add_to(smiu_map)

In [43]:
smiu_map